# Quick Backtest Tutorial

**Goal**: Run your first backtest in under 10 minutes and understand the results.

This notebook demonstrates:
- Loading configuration from YAML files
- Running a 1-day backtest with the HedgeGridV1 strategy
- Analyzing performance metrics
- Visualizing trades and equity curve

**Prerequisites**:
- Python 3.11+ with uv installed
- Historical data in `data/catalog/` (see `verify_data_pipeline.py`)
- 5-10 minutes of your time

**Note**: This uses a short 1-day backtest for demonstration. For meaningful results, test on ≥6 months of data.

## Cell 1: Imports and Setup

In [ ]:
"""Import required packages and configure notebook."""

import sys
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from rich.console import Console
from rich.table import Table

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Import naut-hedgegrid components
from naut_hedgegrid.config.backtest import BacktestConfig  # noqa: E402
from naut_hedgegrid.config.strategy import StrategyConfig  # noqa: E402
from naut_hedgegrid.runners.run_backtest import run_backtest  # noqa: E402

# Configure notebook
%matplotlib inline
plt.style.use("seaborn-v0_8-darkgrid")
console = Console()

console.print("[green]✓ Imports successful[/green]")

## Cell 2: Check Prerequisites

Verify that all required files and data are present before running the backtest.

In [ ]:
"""Check that prerequisites are met."""

# Define paths
data_dir = project_root / "data" / "catalog"
config_dir = project_root / "configs"
backtest_config_path = config_dir / "backtest" / "btcusdt_mark_trades_funding.yaml"
strategy_config_path = config_dir / "strategies" / "hedge_grid_v1.yaml"

# Check data directory
if not data_dir.exists():
    console.print(f"[red]✗ Data directory not found: {data_dir}[/red]")
    console.print("[yellow]Run examples/verify_data_pipeline.py to set up data[/yellow]")
else:
    console.print(f"[green]✓ Data directory found: {data_dir}[/green]")

# Check config files
for config_path in [backtest_config_path, strategy_config_path]:
    if not config_path.exists():
        console.print(f"[red]✗ Config not found: {config_path}[/red]")
    else:
        console.print(f"[green]✓ Config found: {config_path.name}[/green]")

console.print("\n[bold]Ready to run backtest![/bold]")

## Cell 3: Load Configuration Files

Load the backtest and strategy configuration from YAML files.

In [ ]:
"""Load backtest and strategy configurations."""

# Load backtest config
backtest_config = BacktestConfig.load(backtest_config_path)
console.print(
    f"[cyan]Backtest Period:[/cyan] {backtest_config.start_time} to {backtest_config.end_time}"
)
console.print(f"[cyan]Data Sources:[/cyan] {len(backtest_config.data_sources)} sources")

# Load strategy config
strategy_config = StrategyConfig.load(strategy_config_path)
console.print(f"[cyan]Strategy:[/cyan] {strategy_config.class_name}")
console.print(f"[cyan]Grid Levels:[/cyan] {strategy_config.params.get('grid_levels', 'N/A')}")
console.print(f"[cyan]Grid Step:[/cyan] {strategy_config.params.get('grid_step_bps', 'N/A')} bps")

console.print("\n[green]✓ Configuration loaded successfully[/green]")

## Cell 4: Run Backtest

Execute the backtest with the loaded configuration. This may take 30-60 seconds depending on data size.

**What's happening**:
1. NautilusTrader BacktestEngine initializes
2. Historical data loaded from Parquet catalog
3. HedgeGridV1 strategy receives bar data
4. Orders generated, matched, and filled
5. Performance metrics calculated
6. Results saved to `artifacts/` directory

In [ ]:
"""Run the backtest and capture results."""

console.print("[yellow]Running backtest... (this may take 30-60 seconds)[/yellow]\n")

# Run backtest
try:
    results = run_backtest(
        backtest_config=backtest_config,
        strategy_config=strategy_config,
        output_dir=project_root / "artifacts" / "notebook_demo",
    )
    console.print("[green]✓ Backtest completed successfully![/green]")
except Exception as e:
    console.print(f"[red]✗ Backtest failed: {e}[/red]")
    raise

## Cell 5: Extract Results

Parse the backtest results and extract key metrics.

In [ ]:
"""Extract key metrics from backtest results."""

# Extract performance metrics
metrics = results.get("metrics", {})
orders = results.get("orders", [])
positions = results.get("positions", [])
equity_curve = results.get("equity_curve", [])

# Calculate summary statistics
total_orders = len(orders)
total_positions = len(positions)
filled_orders = len([o for o in orders if o.get("status") == "FILLED"])

console.print(f"[cyan]Total Orders:[/cyan] {total_orders}")
console.print(f"[cyan]Filled Orders:[/cyan] {filled_orders}")
console.print(f"[cyan]Total Positions:[/cyan] {total_positions}")

console.print("\n[green]✓ Results extracted[/green]")

## Cell 6: Display Key Performance Indicators (KPIs)

Present the most important metrics in a formatted table.

In [ ]:
"""Display KPIs in a Rich table."""

# Create KPI table
table = Table(title="Backtest Performance Summary", show_header=True, header_style="bold cyan")
table.add_column("Metric", style="cyan", width=30)
table.add_column("Value", style="magenta", justify="right")

# Add key metrics
kpis = [
    ("Total Return", f"{metrics.get('total_return', 0):.2%}"),
    ("Sharpe Ratio", f"{metrics.get('sharpe_ratio', 0):.2f}"),
    ("Sortino Ratio", f"{metrics.get('sortino_ratio', 0):.2f}"),
    ("Max Drawdown", f"{metrics.get('max_drawdown', 0):.2%}"),
    ("Win Rate", f"{metrics.get('win_rate', 0):.2%}"),
    ("Profit Factor", f"{metrics.get('profit_factor', 0):.2f}"),
    ("Total Trades", f"{total_positions}"),
    ("Winning Trades", f"{metrics.get('winning_trades', 0)}"),
    ("Losing Trades", f"{metrics.get('losing_trades', 0)}"),
    ("Avg Win", f"${metrics.get('avg_win', 0):.2f}"),
    ("Avg Loss", f"${metrics.get('avg_loss', 0):.2f}"),
    ("Largest Win", f"${metrics.get('largest_win', 0):.2f}"),
    ("Largest Loss", f"${metrics.get('largest_loss', 0):.2f}"),
]

for metric, value in kpis:
    table.add_row(metric, value)

console.print(table)

# Add interpretation
console.print("\n[bold]Interpretation Guide:[/bold]")
console.print("• [green]Sharpe > 1.0[/green]: Good risk-adjusted returns")
console.print("• [green]Win Rate > 50%[/green]: More winners than losers")
console.print("• [green]Profit Factor > 1.5[/green]: Strong edge")
console.print("• [yellow]Max Drawdown < 20%[/yellow]: Acceptable risk")
console.print("\n[yellow]⚠️  Remember: This is a 1-day demo.[/yellow]")
console.print("[yellow]Test on ≥6 months for meaningful results.[/yellow]")

## Cell 7: Visualize Equity Curve

Plot the equity curve to see account balance over time.

In [ ]:
"""Plot equity curve."""

if equity_curve:
    # Convert to DataFrame
    df_equity = pd.DataFrame(equity_curve)
    df_equity["timestamp"] = pd.to_datetime(df_equity["timestamp"])
    df_equity = df_equity.set_index("timestamp")

    # Create plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df_equity.index, df_equity["equity"], linewidth=2, label="Equity", color="#2E86AB")
    ax.fill_between(df_equity.index, df_equity["equity"], alpha=0.3, color="#2E86AB")

    # Add horizontal line at starting equity
    starting_equity = df_equity["equity"].iloc[0]
    ax.axhline(y=starting_equity, color="gray", linestyle="--", alpha=0.5, label="Starting Equity")

    # Formatting
    ax.set_xlabel("Time", fontsize=12)
    ax.set_ylabel("Equity ($)", fontsize=12)
    ax.set_title("Equity Curve", fontsize=14, fontweight="bold")
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    console.print("[green]✓ Equity curve plotted[/green]")
else:
    console.print("[yellow]⚠ No equity curve data available[/yellow]")

## Cell 8: Visualize Trade Distribution

Show the distribution of trade PnL to understand win/loss patterns.

In [ ]:
"""Plot trade PnL distribution."""

if positions:
    # Extract PnL from positions
    pnl_values = [p.get("realized_pnl", 0) for p in positions if "realized_pnl" in p]

    if pnl_values:
        # Create histogram
        fig, ax = plt.subplots(figsize=(12, 6))

        # Separate wins and losses
        wins = [p for p in pnl_values if p > 0]
        losses = [p for p in pnl_values if p < 0]

        # Plot
        ax.hist(wins, bins=20, alpha=0.7, color="green", label=f"Wins ({len(wins)})")
        ax.hist(losses, bins=20, alpha=0.7, color="red", label=f"Losses ({len(losses)})")

        # Add vertical line at zero
        ax.axvline(x=0, color="black", linestyle="--", linewidth=2, alpha=0.5)

        # Formatting
        ax.set_xlabel("Trade PnL ($)", fontsize=12)
        ax.set_ylabel("Frequency", fontsize=12)
        ax.set_title("Trade PnL Distribution", fontsize=14, fontweight="bold")
        ax.legend()
        ax.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

        console.print("[green]✓ Trade distribution plotted[/green]")
    else:
        console.print("[yellow]⚠ No PnL data available in positions[/yellow]")
else:
    console.print("[yellow]⚠ No position data available[/yellow]")

## Cell 9: Analyze Order Flow

Visualize the order flow over time to understand trading activity.

In [ ]:
"""Analyze order flow patterns."""

if orders:
    # Convert to DataFrame
    df_orders = pd.DataFrame(orders)

    if "timestamp" in df_orders.columns and "side" in df_orders.columns:
        df_orders["timestamp"] = pd.to_datetime(df_orders["timestamp"])
        df_orders["hour"] = df_orders["timestamp"].dt.hour

        # Count orders by hour and side
        grouped = df_orders.groupby(["hour", "side"]).size().reset_index(name="count")
        order_counts = grouped.pivot_table(
            index="hour", columns="side", values="count", fill_value=0
        )

        # Create plot
        fig, ax = plt.subplots(figsize=(12, 6))

        if "BUY" in order_counts.columns:
            ax.bar(order_counts.index, order_counts["BUY"], alpha=0.7, color="green", label="BUY")
        if "SELL" in order_counts.columns:
            ax.bar(
                order_counts.index,
                order_counts["SELL"],
                alpha=0.7,
                color="red",
                label="SELL",
                bottom=order_counts.get("BUY", 0),
            )

        # Formatting
        ax.set_xlabel("Hour of Day", fontsize=12)
        ax.set_ylabel("Number of Orders", fontsize=12)
        ax.set_title("Order Flow by Hour", fontsize=14, fontweight="bold")
        ax.legend()
        ax.grid(True, alpha=0.3, axis="y")
        ax.set_xticks(range(24))

        plt.tight_layout()
        plt.show()

        console.print("[green]✓ Order flow analyzed[/green]")
    else:
        console.print("[yellow]⚠ Required columns not found in order data[/yellow]")
else:
    console.print("[yellow]⚠ No order data available[/yellow]")

## Cell 10: Summary and Next Steps

Congratulations! You've completed your first backtest. Here's what to do next.

In [ ]:
"""Display summary and next steps."""

console.print("\n" + "=" * 70)
console.print("[bold cyan]Congratulations! You've completed your first backtest.[/bold cyan]")
console.print("=" * 70 + "\n")

console.print("[bold]What you learned:[/bold]")
console.print("✓ How to load configuration from YAML files")
console.print("✓ How to run a backtest with NautilusTrader")
console.print("✓ How to analyze performance metrics")
console.print("✓ How to visualize trading results\n")

console.print("[bold]Next steps:[/bold]")
console.print("\n[cyan]1. Extend the backtest period[/cyan]")
console.print("   • Edit configs/backtest/btcusdt_mark_trades_funding.yaml")
console.print("   • Set start_time and end_time to cover ≥6 months")
console.print("   • Re-run this notebook\n")

console.print("[cyan]2. Modify strategy parameters[/cyan]")
console.print("   • Edit configs/strategies/hedge_grid_v1.yaml")
console.print("   • Adjust grid_step_bps, grid_levels, base_qty")
console.print("   • Test different parameter combinations\n")

console.print("[cyan]3. Try paper trading[/cyan]")
console.print("   • Run: uv run python -m naut_hedgegrid paper --enable-ops")
console.print("   • Test with live market data (no real money)")
console.print("   • Compare results to backtest (see tests/test_parity.py)\n")

console.print("[cyan]4. Set up monitoring[/cyan]")
console.print("   • Start paper trading with Prometheus metrics")
console.print("   • Visit http://localhost:9090/metrics")
console.print("   • Set up Grafana dashboards\n")

console.print("[cyan]5. Implement a kill switch[/cyan]")
console.print("   • See examples/kill_switch_integration.py")
console.print("   • Set up PnL/drawdown alerts")
console.print("   • Test emergency position flattening\n")

console.print("[bold yellow]⚠️  IMPORTANT REMINDER[/bold yellow]")
console.print("• This was a [bold]1-day demo backtest[/bold] - not statistically significant")
console.print("• Always test on [bold]≥6 months[/bold] of historical data")
console.print("• Run [bold]paper trading for ≥1 week[/bold] before considering live")
console.print("• [bold red]Never skip paper trading[/bold red]")
console.print("• Read the [bold]Safety Notes[/bold] section in README.md\n")

console.print("[bold green]Happy Trading! 🚀[/bold green]\n")